In [11]:
import os 
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = True
PATH = ''
class DogsVSCats():
    IMG_SIZE = 50
    CATS = "../../../../../data/PetImages/Cat"
    DOGS = "../../../../../data/PetImages/Dog"
    LABELS = {CATS: 0
             ,DOGS: 1}
    training_data = []
    catcount = 0
    dogcount = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label,f)
                    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    
                    img = cv2.resize(img,(self.IMG_SIZE,self.IMG_SIZE))

                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]]) #convert scalar value to one hot vector.
                    
                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                except:
                    pass
        np.random.shuffle(self.training_data)
        np.save("training_data.npy",self.training_data)
        print("Cats: ",self.catcount)
        print("Dogs: ",self.dogcount)
        
if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()
    
training_data = np.load("training_data.npy",allow_pickle=True)
print(len(training_data))

../../../../../data/PetImages/Cat


FileNotFoundError: [Errno 2] No such file or directory: '../../../../../data/PetImages/Cat'

In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) #nr inputs, nr output features, kernel/window size (to roll over features).
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        
        self._to_linear = None #variable for internal use only!
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512,2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) #(2,2) is the shape of the pooling
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        #print(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
load_model = False

if load_model:
    net = Net()
else: 
    net = TheModelClass(*args, **kwargs)
    net.load_state_dict(torch.load(PATH))
    net.eval()

In [ ]:
#check if the gpu is available, use it if so.
torch.cuda.is_available()
torch.device("cuda:0") #in case we have multiple GPU's, select the first one
device

#don't assume everyone has a GPU.
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU.")
else:
    device = torch.device("cpu")
    print("Running on the CPU.")
    
torch.cuda.device_count() #how many GPU's do we have? If we have multiple, we could dispense the workload of the NN across the multiple GPU's.
net = Net().to(device)

In [4]:
import torch.optim as optim

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0 #scale the images to (0,1) range
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

2494


In [5]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [ ]:
def train(net, save_model = False):   
    optimizer = optim.Adam(net.parameters(),lr=0.001)
    loss_function = nn.MSELoss()
    
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
          #  print(i, i+BATCH_SIZE)
            batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
            batch_y = train_y[i:i+BATCH_SIZE]
            
            batch_X, batch_y = batch.to(device), batch_y.to(device)
            
            net.zero_grad() #use zero_grad from nn.Module, could also use optimizer.zero_grad(), which uses the zero_grad from the Adam optimizer
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
    print(f"Epoch {}, Loss: {}".format(epoch, loss))
    
    if save_model:
        torch.save(net.state_dict(), PATH)
        
#assume we don't have the entire train set on the GPU, this is impractical for larger datasets.
train(net)


In [ ]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i]).view(-1, 1, 50, 50).to(device))[0]
            predicted_class = torch.argmax(net_out)
            
            if predicted_class == real_class:
                correct+=1
            total +=1
    print("Accuracy: ", round(correct/total,3))
    
test(net)

In [ ]:
#we should check how in-sample accuracy compares to out-sample accuracy, and the loss should continue to fall.